In [ ]:
import time
from pandas import DataFrame
from ipywidgets import Text, Button, Label, HBox, Output
from IPython.display import display
from collections import defaultdict


class Employee:
    def __init__(self, name: str, salary: float) -> None:
        self.name = name
        self.salary = salary


class Application:
    def __init__(self, employees: list[Employee]) -> None:
        self._employees = employees
        self._container: HBox|None = None
        self._keyword_textbox: Text|None = None
        self._search_button: Button|None = None
        self._status: Label|None = None
        self._output: Output|None = None
        self._initialize_view()

    def _initialize_view(self):
        self._keyword_textbox = Text(value='',
                                     placeholder='请输入关键字',
                                     description='关键字')
        self._search_button = Button(description='查询',
                                     button_style='info',
                                     tooltip='查询',
                                     icon='search')
        self._status = Label()
        self._container = HBox((self._keyword_textbox, self._search_button, self._status))

        self._output = Output()
        self._bind_event()

    def _query_employees(self) -> dict[str, list[str, float]]:
        result = defaultdict(list)

        for employee in self._employees:
            if self._keyword_textbox.value.lower() in employee.name.lower():
                result['name'].append(employee.name)
                result['salary'].append(employee.salary)

        # 模拟获取数据延迟
        time.sleep(1)

        return result

    def _on_click(self, *args):
        # 显示进度信息
        self._status.value = '正在读取数据...'
        # 获取数据
        data = self._query_employees()
        # 隐藏进度信息
        self._status.value = ''
        # 绑定信息
        df = DataFrame(data)
        # 呈现列表
        with self._output:
            self._output.clear_output()
            display(df)

    def _bind_event(self):
        self._search_button.on_click(self._on_click)

    def run(self):
        display(self._container, self._output)


data = [('Andy', 99999.9), ('Tracy', 55555.8), ('Antony', 12345.9)]
employees = [Employee(*entry) for entry in data]

app = Application(employees)
app.run()

In [ ]:
from abc import abstractmethod, ABC


class EmployeeRepository(ABC):
    @abstractmethod
    def query_employees(self, keyword: str) -> list[Employee]:
        pass


class Service(ABC):
    @abstractmethod
    def query_employees_data(self, keyword: str):
        pass


class View(ABC):
    @property
    @abstractmethod
    def employee_presenter(self):
        pass

    @employee_presenter.setter
    @abstractmethod
    def employee_presenter(self, presenter):
        pass

    @abstractmethod
    def show_employees(self, data: dict[str, list[str, float]]) -> None:
        pass

    @abstractmethod
    def display(self) -> None:
        pass

    @abstractmethod
    def show_status(self) -> None:
        pass

    @abstractmethod
    def hide_status(self) -> None:
        pass


class Presenter(ABC):
    @property
    @abstractmethod
    def employee_view(self):
        pass

    @employee_view.setter
    @abstractmethod
    def employee_view(self, view):
        pass

    @abstractmethod
    def search(self, keyword: str) -> None:
        pass

    @abstractmethod
    def display(self) -> None:
        pass


In [ ]:
import time
from ipywidgets import Text, Button, Label, HBox, Output
from IPython.display import display
from collections import defaultdict

from pandas import DataFrame


class Employee:
    def __init__(self, name: str, salary: float) -> None:
        self.name = name
        self.salary = salary


class MemoryEmployeeRepository(EmployeeRepository):
    employees: list[Employee] = [
        Employee('Andy', 99999.9),
        Employee('Tracy', 44444.4),
        Employee('Antony', 12345.8)
    ]

    def query_employees(self, keyword: str) -> list[Employee]:
        return [employee for employee in self.employees
                if keyword.lower() in employee.name.lower()]


class EmployeeService(Service):
    def __init__(self, employee_repository: EmployeeRepository):
        self._employee_repository = employee_repository

    def query_employees_data(self, keyword: str) -> dict[str, list[str, float]]:
        employees = self._employee_repository.query_employees(keyword)
        result = defaultdict(list)
        for employee in employees:
            result['name'].append(employee.name)
            result['salary'].append(employee.salary)

        return result


class EmployeePresenter(Presenter):
    def __init__(self, employee_service: Service) -> None:
        self._employee_service = employee_service
        self._employee_view = None

    @property
    def employee_view(self):
        return self._employee_view

    @employee_view.setter
    def employee_view(self, view):
        self._employee_view = view

    def search(self, keyword: str) -> None:
        # 1.显示进度信息
        self.employee_view.show_status()
        # 2.获取数据
        data = self._employee_service.query_employees_data(keyword)
        # 模拟延迟
        time.sleep(0.5)
        # 3.隐藏进度信息
        self.employee_view.hide_status()
        # 3.绑定和呈现列表
        self.employee_view.show_employees(data)

    def display(self) -> None:
        self.employee_view.display()


class EmployeeView(View):
    def __init__(self):
        self._employee_presenter: EmployeePresenter | None = None
        self._container: HBox | None = None
        self._keyword_textbox: Text | None = None
        self._search_button: Button | None = None
        self._status: Label | None = None
        self._output: Output | None = None
        self._initialize_view()

    def _initialize_view(self) -> None:
        self._keyword_textbox = Text(value='',
                                     placeholder='请输入关键字',
                                     description='关键字')
        self._search_button = Button(description='查询',
                                     button_style='info',
                                     tooltip='查询',
                                     icon='search')
        self._status = Label()
        self._container = HBox((self._keyword_textbox, self._search_button, self._status))

        self._output = Output()
        self._bind_event()

    def _on_click(self, *args):
        keyword = self._keyword_textbox.value
        self.employee_presenter.search(keyword)

    def _bind_event(self):
        self._search_button.on_click(self._on_click)

    @property
    def employee_presenter(self):
        return self._employee_presenter

    @employee_presenter.setter
    def employee_presenter(self, presenter):
        self._employee_presenter = presenter

    def show_employees(self, data: dict[str, list[str, float]]) -> None:
        df = DataFrame(data)

        with self._output:
            self._output.clear_output()
            display(df)

    def display(self) -> None:
        display(self._container, self._output)

    def show_status(self) -> None:
        self._status.value = '正在读取数据...'

    def hide_status(self) -> None:
        self._status.value = ''


if __name__ == '__main__':
    view = EmployeeView()
    repository = MemoryEmployeeRepository()
    service = EmployeeService(repository)
    presenter = EmployeePresenter(service)

    presenter.employee_view = view
    view.employee_presenter = presenter


    presenter.display()
